## Preprocessing

In [39]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [40]:
# I've been running this on my laptop, which turns out to be faster than (free) Google Colab,
# and this shows whether I have it set up right to use the GPU:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [41]:
# Constants to toggle adjustments I've made to the cleanup
DROP_STATUS = True
DROP_INCOME_AMT = True
DROP_ASK_AMT = True

ADD_INCOME_MID = True
ADD_INCOME_ASK_RATIO = True

APPLICATION_CUTOFF = 50     # Originally...
CLASSIFICATION_CUTOFF = 100 # Originally...

# Whether to run the tuner or the hard-coded network build code
TUNE_NETWORK = False
BUILD_NETWORK = !TUNE_NETWORK

In [42]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
del application_df["EIN"]
del application_df["NAME"]

# Also drop STATUS since there only 5 non-active rows
if DROP_STATUS:
    del application_df["STATUS"]

In [43]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [44]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# Disabled because I'm tired of scrolling past it:
# v_counts = application_df["APPLICATION_TYPE"].value_counts()
# v_counts

In [45]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = v_counts[v_counts < APPLICATION_CUTOFF].index.values

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
# Disabled because I'm tired of scrolling past it:
#application_df['APPLICATION_TYPE'].value_counts()

In [46]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
# Disabled because I'm tired of scrolling past it:
#v_counts = application_df["CLASSIFICATION"].value_counts()
#v_counts

In [47]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# Disabled because I'm tired of scrolling past it:
# v_counts[v_counts > 1]

In [48]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = v_counts[v_counts < CLASSIFICATION_CUTOFF].index.values

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [49]:
# The INCOME_AMT column is categorical but could be numerical, which might be helpful.
if ADD_INCOME_MID:
    application_df["INCOME_MID"] = application_df["INCOME_AMT"].map({
        "0": 0,
        "25000-99999": 62499.5,
        "100000-499999": 299999.5,
        "1M-5M": 3000000,
        "1-9999": 5000,
        "10000-24999": 17499.5,
        "10M-50M": 30000000,
        "5M-10M": 7500000,
        "50M+": 50000000})
    print(application_df["INCOME_MID"].value_counts())

if DROP_INCOME_AMT:
    del application_df["INCOME_AMT"]


INCOME_MID
0.0           24388
62499.5        3747
299999.5       3374
3000000.0       955
5000.0          728
17499.5         543
30000000.0      240
7500000.0       185
50000000.0      139
Name: count, dtype: int64


In [50]:
# Supplement the ask and income columns with a ratio
if ADD_INCOME_ASK_RATIO:
    application_df["INCOME_ASK_RATIO"] = application_df["INCOME_MID"] / application_df["ASK_AMT"]

# Surprisingly, the overwhelming about of loans are asking for the same amount of money, $5000,
# which could badly skew the model. So drop that column.
if DROP_ASK_AMT:
    del application_df["ASK_AMT"]

In [51]:
# Change the SPECIAL_CONSIDERATIONS column into something more easily usable
application_df["SPECIAL_CONSIDERATIONS"] = application_df["SPECIAL_CONSIDERATIONS"].map({"Y":1, "N":0})

In [52]:
# Convert categorical data to numeric with `pd.get_dummies`
cat_columns = ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION"]

if not DROP_INCOME_AMT:
    cat_columns.append("INCOME_AMT")

dummied_df = pd.get_dummies(application_df, columns=cat_columns)
dummied_df.head()

,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,INCOME_MID,INCOME_ASK_RATIO,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,...,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,0,1,0.0,0.000000,True,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False
1,0,1,5000.0,0.046045,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,False
2,0,0,0.0,0.000000,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False
3,0,1,17499.5,2.614988,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
4,0,1,299999.5,2.103931,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [53]:
# Split our preprocessed data into our features and target arrays
X = dummied_df.loc[:, dummied_df.columns != "IS_SUCCESSFUL"]
y = dummied_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8675309)
print(X.shape, y.shape, X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Dig in to the breakdown of training vs test sets

(34299, 47) (34299,) (25724, 47) (8575, 47) (25724,) (8575,)


In [54]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((25724, 47), (8575, 47))

## Hyperband Search for Model Structure

In [55]:
%%time

MAX_EPOCHS_PER_MODEL = 7 # Meant to get a decent idea of parameter, not create a final model
HYPERBAND_ITERATIONS = 1  # "Number of times to iterate over the full Hyperband algorithm"
EXECUTIONS_PER_TRIAL = 2  # Training from scratch
SEARCH_FIT_EPOCHS = 5    # Epochs for each attempt to do a fit, I think. Not sure how this relates to MAX_EPOCHS_PER_MODEL.
OVERWRITE = True          # I'm hoping to be able to interrupt a run and resume it later

#ACTIVATIONS = ["tanh", "relu", "elu", "selu", "exponential", "gelu", "mish", "relu6", "silu"]
#LAYER_COUNTS = [6, 12]
#UNITS_PER_LAYER_COUNTS = [30, 60]

#ACTIVATIONS = ["tanh", "relu", "elu", "selu", "exponential", "gelu", "mish", "relu6", "silu"] -> Favorite: gelu
#ACTIVATIONS = ["tanh", "relu", "elu", "selu"]                                                 -> Favorite: tanh (surprise!)
#ACTIVATIONS = ["exponential", "mish", "relu6", "silu"]                                        -> Favorites: relu6 or mish, silu

ACTIVATIONS = ["gelu", "tanh", "relu6", "mish", "silu"]
LAYER_COUNTS = [2]
UNITS_PER_LAYER_COUNTS = [30]

# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):

    nn = tf.keras.models.Sequential()

    # Input layer
    in_shape = (X.shape[1],)
    nn.add(tf.keras.Input(shape=in_shape))    

    # Hidden layers
    layer_count = hp.Choice(f'layer_count', LAYER_COUNTS)
    for l in range(layer_count):
        # Select activation and number of units for this layer
        activation = hp.Choice(f'activation_{l:04}', ACTIVATIONS)
        units =      hp.Choice(f'units______{l:04}', UNITS_PER_LAYER_COUNTS)

        # Add this layer
        nn.add(tf.keras.layers.Dense(units=units, activation=activation))
        


    # Output layer    
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn


# Run the kerastuner search for best hyperparameters
if TUNE_NETWORK:
    tuner = kt.Hyperband(
        create_model,
        objective="val_accuracy",
        max_epochs=MAX_EPOCHS_PER_MODEL,
        hyperband_iterations=HYPERBAND_ITERATIONS,
        executions_per_trial=EXECUTIONS_PER_TRIAL,
        overwrite=OVERWRITE)
    tuner.search(X_train_scaled,y_train,epochs=SEARCH_FIT_EPOCHS,validation_data=(X_test_scaled,y_test))
    
    best_hyper = tuner.get_best_hyperparameters(1)[0]
    print(f"Best Hyper Values: {best_hyper.values}")
    
    best_model = tuner.get_best_models(1)[0]
    model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Best Model: Loss: {model_loss}, Accuracy: {model_accuracy}")


CPU times: user 12 μs, sys: 4 μs, total: 16 μs
Wall time: 17.9 μs


['exponential', 'mish', 'relu6', 'silu']

## Compile, Train and Evaluate the Model

In [56]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
if BUILD_NETWORK:
    in_shape = (X.shape[1],)
    nn = tf.keras.models.Sequential()
    
    # Input layer
    nn.add(tf.keras.Input(shape=in_shape))
    
    As = ["tanh", "gelu", "mish"]
    us = [30]
    
    for u in us:
        for a in As:
            nn.add(tf.keras.layers.Dense(units=u, activation=a))
    
    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    
    # Check the structure of the model
    nn.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 30)             │         1,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 30)             │           930 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 30)             │           930 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,331 (13.01 KB)

 Trainable params: 3,331 (13.01 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
# Set up training checkpoint to save after each epoch, if it is a new best model:
checkpoint_filepath = './best.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='valaccuracy',
    mode='max',
    save_best_only=True,
    save_weights_only=False,
    verbose=1)  


In [61]:
%%time

# Running through regular Windows, laptop CPU
# 5 epochs:
# CPU times: total: 20.7 s
# Wall time: 31.2 s
# 25 epochs:
# CPU times: total: 1min 14s
# Wall time: 2min 7s

# Running through WSL, laptop GPU
# 5 epochs:
# CPU times: user 21.7 s, sys: 3.82 s, total: 25.5 s
# Wall time: 19.2 s
# 25 epochs:
# CPU times: user 1min 12s, sys: 15.1 s, total: 1min 27s
# Wall time: 1min 18s

# Google Colab, free / default resourcing, reported as CPU
# 5 epochs:
# CPU times: user 29.6 s, sys: 730 ms, total: 30.3 s
# Wall time: 36.3 s
# 25 epochs:
# CPU times: user 1min 46s, sys: 3.33 s, total: 1min 50s
# Wall time: 2min 5s

EPOCHS = 5
if BUILD_NETWORK:    
    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])    
    
    # Train the model
    nn.fit(X_train_scaled, y_train, epochs=EPOCHS, callbacks=[model_checkpoint_callback])
    
    model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Final Model, Test Set results: Loss: {model_loss}, Accuracy: {model_accuracy}")
    
    model_loss, model_accuracy = tf.keras.models.load_model(checkpoint_filepath).evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Best  Model, Test Set results: Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/5
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7349 - loss: 0.5441
Epoch 2/5
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7323 - loss: 0.5456
Epoch 3/5
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7309 - loss: 0.5497
Epoch 4/5
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7363 - loss: 0.5402
Epoch 5/5
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7359 - loss: 0.5436
268/268 - 1s - 3ms/step - accuracy: 0.7354 - loss: 0.5480
Final Model, Test Set results: Loss: 0.5480361580848694, Accuracy: 0.7353935837745667


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_21" is incompatible with the layer: expected axis -1 of input shape to have value 42, but received input with shape (None, 47)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 47), dtype=float32)
  • training=False
  • mask=None

In [59]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")